# Analisi dati di allenamento Garmin

Disponendo di un ampio dataset con oltre 75.000 registrazioni provenienti da un dispositivo Garmin (Edge 820), relative ad allenamenti in bicicletta, l'obiettivo è sviluppare modelli predittivi basati su alcuni parametri (come la velocità, la pendenza, ecc.) per prevedere alcuni parametri di interesse.
- *Regressione*:
    - predire il battito cardiaco istantaneo (bpm)
    - predire la potenza istantanea (W)
- *Classificazione*:
    - classificare la zona di frequenza cardiaca
    - classificare la zona di potenza

## Raccolta dei dati
Poiché i dati non sono stati raccolti da siti noti come [UCI](https://archive.ics.uci.edu/), [Kaggle](https://www.kaggle.com/), ecc. bensì da allenamenti svolti personalmente da Riccardo Fiorani, è stato necessario estrarre i dati dai file *.fit* e *.tcx* messi a disposizione da Garmin e convertirli in *.csv* in modo da poterli analizzare con Python.
Per fare questo si è utilizzato il tool [Fit File Repair Tool](https://www.fitfilerepairtool.info/), che permette di convertire i file *.fit* in *.csv*.
Aggiungi ulteriore descrizione....

## Descrizione dei dati
Non essendo i dati provenienti da un dataset noto, è necessario descriverli in modo da poterli analizzare e comprendere meglio.
I *CSV* e i formati originali sono visionabili nella cartella del progetto `garmin_edge_820/`.
In particolare il dataset utilizzato è dato dall'unione di 4 allenamenti, ciascuno relativo a un allenamento.
Ogni record rappresenta un istante di un dato allenamento, raccolto con cadenza di un secondo, e contiene le seguenti informazioni:
- `timestamp`: data e ora dell'istante
- `distance`: distanza percorsa dall'inizio dell'allenamento (m)
- `accumulated_power`: potenza accumulata da inizio allenamento (W)
- `altitude`: altitudine (m)
- `speed`: velocità istantanea (m/s)
- `power`: potenza istantanea (W)
- `heart_rate`: frequenza cardiaca (bpm)
- `cadence`: cadenza di pedalata (rpm)
- `temperature`: temperatura (°C)
- `left_right_balance`: bilanciamento destro/sinistro, corrisponde alla percentuale di potenza applicata dal piede destro in rapporto a quella applicata dal piede sinistro. 
- `left_pco`: parametro Garmin che indica la distribuzione della forza lungo la battuta del pedale durante la pedalata. In questo caso relativo al piede sinistro. Il PCO si misura in millimetri. Valori positivi (ad es. +6 mm) indicano una forza maggiore verso l'esterno del pedale, mentre valori negativi (ad es. -4 mm) indicano una maggiore forza verso l'interno del pedale. Si veda la seguente immagine per ulteriori informazioni
<div align="center"><img src="imgs/pco.png" width="200"></div>

- `right_pco`: come sopra ma relativo al piede destro.
- `left_power_phase`: I sensori di potenza Garmin rilevano il punto in cui la gamba genera una coppia positiva in una pedalata e dove si verifica la maggiore concentrazione di coppia positiva. Inoltre, rilevano l'angolo a cui queste forze iniziano e terminano e il punto in cui si produce la concentrazione di potenza.
<div align="center"><img src="imgs/pwrphase.png" width="200"></div>

- `left_power_phase_peak`: picco massimo di power phase sinistra.
- `right_power_phase`: power phase destro.
- `right_power_phase_peak`: picco massimo di power phase destra.

A questi parametri sono stati aggiunti altri parametri derivati, come:
- `time_since_start`: tempo trascorso dall'inizio dell'allenamento (s). Volevamo verificare se vi è una correlazione fra tempo trascorso e frequenza cardiaca, potenza, ecc.
- `hr_zone`: zona di frequenza cardiaca. Per calcolarla abbiamo usato i seguenti intervalli suggeriti da Garmin 
    - Zona 1: (0, 128)
    - Zona 2: (129, 146)
    - Zona 3: (147, 156)
    - Zona 4: (157, 165)
    - Zona 5: (166, 174)
    - Zona 6: (175, 179)
    - Zona 7: $\geq$ 180
- `pwr_zone`: zona di potenza. Per calcolarla abbiamo usato i seguenti intervalli suggeriti da Garmin 
    - Zona 1: (0, 157)
    - Zona 2: (158, 186)
    - Zona 3: (187, 200)
    - Zona 4: (201, 218)
    - Zona 5: (219, 247)
    - Zona 6: (248, 287)
    - Zona 7: $\geq$ 288
- `altitude_diff`: differenza di altitudine rispetto al secondo precedente (m)
- `distance_diff`: differenza di distanza rispetto al secondo precedente (m)
- `slope_percent`: pendenza istantanea (%), calcolata come 
$$
\frac{\text{\texttt{altitude\_diff}}}{\text{\texttt{distance\_diff}}}
$$

In totale il dataset è composto da 77.620 record.

## Preparazione dei dati

Di seguito sono riportate le operazioni di preparazione dei dati effettuate prima di procedere con l'analisi vera e propria.

In [ ]:
# %pip install numpy
# %pip install pandas
# %pip install matplotlib
# %pip install scikit-learn
# %pip install lightgbm
# %pip install xgboost
# %pip install flask
import numpy as np
import pandas as pd

In [ ]:
file_list = ["garmin_edge_820/3993730634_ACTIVITY_data.csv",
             "garmin_edge_820/4557226804_ACTIVITY_data.csv",
             "garmin_edge_820/4593452980_ACTIVITY_data.csv",
             "garmin_edge_820/5191513011_ACTIVITY_data.csv",
]
combined_df = pd.concat([pd.read_csv(file, sep=";") for file in file_list], ignore_index=True)

In [ ]:
combined_df.info()

Poiché alcuni modelli (LGBM Regressor) non supportano features aventi caratteri non unicode, abbiamo sostituito le parentesi quadre con delle tonde usando il metodo `convert_brackets`

In [ ]:
def convert_brackets(string):
    return string.replace('[', '(').replace(']', ')')

combined_df.columns = [convert_brackets(col) for col in combined_df.columns]

A questo punto abbiamo deciso di sostituire la colonna `timestamp(s)` in un formato più comodo da gestire, ovvero `datetime`. Per fare questo abbiamo usato il metodo Pandas `to_datetime`. A questo punto abbiamo reso la colonna risultante `time` l'indice del DataFrame.

In [ ]:
combined_df['timestamp(s)'] = combined_df['timestamp(s)'] + 631065600
combined_df['time'] = pd.to_datetime(combined_df.pop('timestamp(s)'), unit='s')
combined_df.set_index("time", inplace=True)

Per questioni di errore di misurazione, abbiamo visto che alcuni record presentano valori di `speed(m/s)` uguali a 0. Non essendo quei record rilevanti per l'analisi, abbiamo deciso di eliminarli. Inoltre, come mostrato nel `details.info()` superiore, esistono molti valori di `speed(m/s)` posti a `NaN`. Abbiamo deciso di eliminare anche questi record.

In [ ]:
combined_df = combined_df[combined_df['speed(m/s)'] != 0]
combined_df = combined_df.dropna(subset=['speed(m/s)'])

Come anticipato nella sezione precedente, abbiamo aggiunto una feature `time_since_start(s)` che indica il tempo trascorso dall'inizio dell'allenamento. 

In [ ]:
combined_df['time_since_start(s)'] = combined_df.groupby(pd.Grouper(freq='D')).cumcount() + 1

Di seguito è invece implementata una funzione che genera le zone di frequenza cardiaca e di potenza.

In [ ]:
hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165),(166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218),(219, 247), (248, 287), (288, float('inf'))]

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone
        
combined_df['hr_zone'] = combined_df['heart_rate(bpm)'].apply(get_zone, zones=hr_zones)
combined_df['pwr_zone'] = combined_df['power(watts)'].apply(get_zone, zones=power_zones)

A questo punto si calcolano le features relative alla differenza di altitudine e distanza dall'istante successivo. Successivamente vengono utilizzate per calcolare la pendenza istantanea.

In [ ]:
combined_df['altitude_diff(m)'] = combined_df['altitude(m)'] - combined_df['altitude(m)'].shift(1)
combined_df['distance_diff(m)'] = combined_df['distance(m)'] - combined_df['distance(m)'].shift(1)
combined_df[['altitude_diff(m)', 'distance_diff(m)']] = combined_df[['altitude_diff(m)', 'distance_diff(m)']].fillna(0)
combined_df['slope_percent'] = np.where(combined_df['distance_diff(m)'] == 0, 0, combined_df['altitude_diff(m)'] / combined_df['distance_diff(m)'] * 100)

Come mostra il grafico seguente il battito cardiaco è molto impreciso su valori bassi. Poiché è altamente iprobabile aver raggiunto 30 bpm durante un allenamento, abbiamo deciso di eliminare i record con frequenza cardiaca inferiore ad una soglia che, visionando il grafico, abbiamo deciso di porre a 80bpm.
Infatti si può notare che nel secondo e terzo allenamento le frequenze vanno anche sotto gli 80bpm ma assumendo valori troppo bassi o non attendibili (linea piatta).

In [ ]:
import matplotlib.pyplot as plt

def plot_data_vs_time(col):
    fig, ax = plt.subplots(nrows=4, sharex=True, figsize=(24, 9))
    for i, (date, data) in enumerate(combined_df.groupby(combined_df.index.date)):
        ax[i].plot(data["time_since_start(s)"], data[col], label=str(date))

    # Opzionale: Aggiungi legenda
    fig.legend()

    fig.text(0.5, 0.04, "Time since Start", ha="center")
    fig.text(0.04, 0.5, col, va="center", rotation="vertical")
    plt.suptitle(f"{col} vs Time since Start for Each Training Day")

    plt.show()

In [ ]:
plot_data_vs_time('heart_rate(bpm)')

In [ ]:
combined_df.drop(combined_df[combined_df['heart_rate(bpm)'] < 80].index, inplace=True)
plot_data_vs_time('heart_rate(bpm)')

Anche la potenza, come mostrato nel grafico seguente, presenta valori molto instabili. In particolare si notano
- molti buchi, cioè linee rette nel grafico, che indicano che il sensore non ha rilevato la potenza
- picchi molto alti susseguiti da picchi molto bassi. Questo è dovuto al fatto che il sensore di potenza è molto sensibile e rileva anche piccole variazioni di potenza.

Per questo motivo abbiamo deciso di calcolare la media mobile con finestra di 3 secondi. Questo ci permette di avere una visione più chiara della potenza durante l'allenamento.

In [ ]:
plot_data_vs_time('power(watts)')

In [ ]:
window_size = 3 
combined_df['avg_power(watts)'] = combined_df['power(watts)'].rolling(window=int(window_size), center=True).mean()
combined_df['avg_power(watts)'] = combined_df['avg_power(watts)'].dropna()

Il parametro `left_right_balance` di Garmin va trattato per visualizzare l'effettiva potenza di spinta da gamba sinistra e destra. In particolare Garmin suggerisce di usare la seguente formula
$$
\text{\texttt{left\_power}} = \text{\texttt{left\_right\_balance}} - 128 \\
\text{\texttt{right\_power}} = 100 - \text{\texttt{left\_power}}
$$


In [ ]:
combined_df['power_left(watts)'] = combined_df['left_right_balance'] - 128
combined_df['power_right(watts)'] = 100 - combined_df['power_left(watts)']

Come ultima cosa vengono rimosse quelle feature inutilizzate per l'analisi.
- `left_right_balance`: non è più necessaria poiché abbiamo calcolato `left_power` e `right_power`
- `left_power_phase_peak`: non è necessario il picco se abbiamo il valore istantaneo
- `right_power_phase_peak`: come sopra
- `left_power_phase`: (***ANCORA DA CALCOLARE***)
- `right_power_phase`: (***ANCORA DA CALCOLARE***)

In [ ]:
combined_df = combined_df.drop(['left_power_phase(degrees)',
                            'left_power_phase_peak(degrees)',
                            'right_power_phase(degrees)',
                            'right_power_phase_peak(degrees)',
                            'left_right_balance'], axis=1)

In [ ]:
combined_df.describe()

In [ ]:
combined_df.info()

Come mostra il `df.info()`, il dataset ripulito presenta 61398 record e 18 features. Resta ancora una cosa da fare: la potenza m

In [ ]:
combined_df['avg_power(watts)'] = combined_df['avg_power(watts)'].fillna(combined_df['avg_power(watts)'].mean())

# Data Visualization

In [ ]:
def plot_params_per_training(params):
    fig, axs = plt.subplots(nrows=4, figsize=(21, 10), sharex=True)

    for i, (date, data) in enumerate(combined_df.groupby(combined_df.index.date)):
        axs[i].set_title(str(date))

        for param in params:
            axs[i].plot(data["time_since_start(s)"], data[param], label=param)

        axs[i].legend()
        axs[i].grid(True)

    plt.xlabel("time_since_start")
    plt.suptitle("Data vs Time since Start for Each Training Day")
    plt.show()

In [ ]:
plot_params_per_training(['power(watts)', 'cadence(rpm)', 'speed(m/s)', 'heart_rate(bpm)', 'altitude(m)'])

# Previsioni

In [ ]:
import sklearn as sk
from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

### Definizione dei modelli

In [ ]:
def get_pipe_grid(type):
    pipelines = {
        "linear": {
            "pipe": Pipeline([
                ("poly", PolynomialFeatures(include_bias=False)),
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "poly__degree": [3,4],
                "std": [StandardScaler()],
            },
            "grid_regressors": [
                {
                    "regressor": [LinearRegression()],
                },
                {
                    "regressor": [Lasso()],
                    "regressor__alpha": [0.01,0.1],
                },
                {
                    "regressor": [Ridge()],
                    "regressor__alpha": [0.01,0.1],
                },
                {
                    "regressor": [ElasticNet()],
                    "regressor__alpha": [0.01,0.1],
                    "regressor__l1_ratio": [0.1],
                },
            ]
        },
        "kernel": {
            "pipe": Pipeline([
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "std": [StandardScaler()],
                "regressor": [KernelRidge()],
                'regressor__alpha': [0.01,0.1],
            },
            "grid_regressors": [
                {
                    "regressor__kernel": ["poly"],
                    'regressor__degree': [4,5],
                },
                {
                    "regressor__kernel": ["rbf"],
                    "regressor__gamma": [0.01,0.1],
                }
            ]
        },
        "tree": {
            "pipe": Pipeline([
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "std": [StandardScaler()],
                "regressor__max_depth": [5,10],
            },
            "grid_regressors": [
                {
                    "regressor": [DecisionTreeRegressor()],
                },
                {
                    "regressor": [RandomForestRegressor()],
                    "regressor__n_estimators": [100,1000],
                },
                {
                    "regressor": [LGBMRegressor()],
                    "regressor__n_estimators": [100,1000],
                    "regressor__learning_rate": [0.01,0.1],
                },
                {
                    "regressor": [XGBRegressor()],
                    "regressor__n_estimators": [100,1000],
                    "regressor__learning_rate": [0.01,0.1],
                }
            ]
        }
    }
    return pipelines[type]["pipe"], [dict(pipelines[type]["grid_common"], **params) for params in pipelines[type]["grid_regressors"]]

In [ ]:
def get_trained_model(type, X_train, y_train, randomized=False):
    pipe, grid = get_pipe_grid(type)
    model = GridSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1) if not randomized else RandomizedSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1)
    model.fit(X_train, y_train)
    return model

## Previsione battito

In [ ]:
X_hr = details.drop(['heart_rate(bpm)','distance(m)','altitude(m)','speed(m/s)','potenza_media','time_since_start','slope_percent','temperature(C)','hr_zone','pwr_zone','altitude_diff','distance_diff','left_pco(mm)','right_pco(mm)','power_left','power_right','accumulated_power(watts)'], axis=1)
y_hr = details['heart_rate(bpm)']
X_train, X_val, y_train, y_val = train_test_split(X_hr, y_hr, test_size=1/3, random_state=42)

### Regressione lineare

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from utilities import print_eval

In [ ]:
liner_models_gs = get_trained_model('linear', X_train, y_train)
linear_models_gs_res = pd.DataFrame(liner_models_gs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, liner_models_gs)
linear_models_gs_res

In [ ]:
"""
linear_models_rs = get_trained_model('linear', X_train, y_train, randomized=True)
linear_models_rs_res = pd.DataFrame(linear_models_rs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, linear_models_rs)
linear_models_rs_res
"""

### Regressione con funzioni kernel

In [ ]:
krm_gs = get_trained_model('kernel', X_train, y_train)
krm_gs_res = pd.DataFrame(krm_gs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, krm_gs)
krm_gs_res

In [ ]:
"""
krm_rs = get_trained_model('kernel', X_train, y_train, randomized=True)
krm_rs_res = pd.DataFrame(krm_rs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, krm_rs)
krm_rs_res
"""

### Alberi di regressione

In [ ]:
tree_gs = get_trained_model('tree', X_train, y_train)
tree_gs_res = pd.DataFrame(tree_gs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, tree_gs, tree=True)
tree_gs_res

In [ ]:
"""
tree_rs = get_trained_model('tree', X_train, y_train, randomized=True)
tree_rs_res = pd.DataFrame(tree_rs.cv_results_).sort_values("mean_test_score", ascending=False)
print_eval(X_val, y_val, tree_rs, tree=True)
tree_rs_res
"""

### Confronto fra modelli

In [ ]:
# Creazione del plot scatter
fig, ax = plt.subplots(figsize=(24, 9))

linear_models = pd.concat([linear_models_gs_res, linear_models_gs_res], axis=0)
scatter_linear = ax.scatter(linear_models['mean_fit_time'], linear_models['mean_test_score'], color='blue', label='Modelli lineari e polinomiali')

krm_models = pd.concat([krm_gs_res, krm_gs_res], axis=0)
scatter_kernel = ax.scatter(krm_models['mean_fit_time'], krm_models['mean_test_score'], color='red', label='Modelli Kernel')

tree_models = pd.concat([tree_gs_res, tree_gs_res], axis=0)
scatter_tree = ax.scatter(tree_models['mean_fit_time'], tree_models['mean_test_score'], color='green', label='Modelli Albero')

ax.set_xlabel('Tempo di addestramento (s)')
ax.set_ylabel('Coefficiente R2')
ax.set_title('Confronto tra i modelli')
ax.legend()
plt.ylim(0, 1.1)
plt.show()


### test predizione

In [ ]:
X_hr.info()

In [ ]:
tree_gs.predict(np.array([280, 110]).reshape(1, -1))

In [ ]:
pd.DataFrame({'Feature': X_hr.columns, 'Weight': tree_gs.best_estimator_.named_steps['regressor'].feature_importances_})

### Reti neurali

In [ ]:
X_hr = details.drop(['heart_rate(bpm)','potenza_media','temperature(C)','hr_zone','pwr_zone','altitude_diff','distance_diff','left_pco(mm)','right_pco(mm)','power_left','power_right','accumulated_power(watts)'], axis=1)
y_hr = details['heart_rate(bpm)']
X_train, X_val, y_train, y_val = train_test_split(X_hr, y_hr, test_size=1/3, random_state=42)

In [ ]:
from sklearn.neural_network import MLPRegressor

pipe = Pipeline([
    ("std", StandardScaler()),
    ("regressor", MLPRegressor())
])

grid = {
    "regressor__hidden_layer_sizes": [(256, 256)],
    "regressor__activation": ["relu"],
    "regressor__max_iter": [2000],
    "regressor__batch_size": [128],
    "regressor__alpha": [0.01]

}

model = GridSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1)
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

In [ ]:
pd.DataFrame(model.cv_results_).sort_values("mean_test_score", ascending=False)

In [ ]:
pd.DataFrame({'Feature': X_hr.columns, 'Weight': model.best_estimator_.named_steps['regressor'].coefs_[0].mean(axis=1)})

In [ ]:
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1} )
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=X_train.shape[1:]))
    num_layers = hp.Int('num_layers', min_value=1, max_value=5, step=1)
    for i in range(num_layers):
        hp_units = hp.Int(f'units_{i}', min_value=32, max_value=1024, step=32)
        model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    hp_learning_rate = hp.Choice('learning_rate', values=[0.1,0.01,0.001])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='mean_absolute_error',metrics=['mean_absolute_error'])

    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_mean_absolute_error',
                     max_epochs=100,
                     factor=5,
                     directory='tensorflow',
                     project_name='garmin_analysis8')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', patience=3)

tuner.search(X_train, y_train, epochs=10,validation_split=1/3, callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connectedlayer is {best_hps.get('num_layers')} 
and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}.""")

model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=10, validation_split=1/3)
val_loss_per_epoch = history.history['val_loss']
best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=1/3)
eval_result = hypermodel.evaluate(X_val, y_val)
print("[test loss, test mae]:", eval_result)

In [ ]:
hypermodel.summary()

In [ ]:
hypermodel.predict(np.array([0,0,9,100,100,0,30]).reshape(1, -1))

## Previsione Potenza

In [ ]:
def get_pipe_grid(type):
    pipelines = {
        "linear": {
            "pipe": Pipeline([
                ("poly", PolynomialFeatures(include_bias=False)),
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "poly__degree": [4,5],
                "std": [StandardScaler()],
            },
            "grid_regressors": [
                {
                    "regressor": [LinearRegression()],
                },
                {
                    "regressor": [Lasso()],
                    "regressor__alpha": [0.01,0.1],
                },
                {
                    "regressor": [Ridge()],
                    "regressor__alpha": [0.01,0.1],
                },
                {
                    "regressor": [ElasticNet()],
                    "regressor__alpha": [0.01,0.1],
                    "regressor__l1_ratio": [0.1],
                },
            ]
        },
        "kernel": {
            "pipe": Pipeline([
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "std": [StandardScaler()],
                "regressor": [KernelRidge()],
                'regressor__alpha': [0.01,0.1],
            },
            "grid_regressors": [
                {
                    "regressor__kernel": ["poly"],
                    'regressor__degree': [4,5],
                },
                {
                    "regressor__kernel": ["rbf"],
                    "regressor__gamma": [0.01,0.1],
                }
            ]
        },
        "tree": {
            "pipe": Pipeline([
                ("std", None),
                ("regressor", None)
            ]),
            "grid_common": {
                "std": [StandardScaler()],
                "regressor__max_depth": [5,10],
            },
            "grid_regressors": [
                {
                    "regressor": [DecisionTreeRegressor()],
                },
                {
                    "regressor": [RandomForestRegressor()],
                    "regressor__n_estimators": [100,1000],
                },
                {
                    "regressor": [LGBMRegressor()],
                    "regressor__n_estimators": [100,1000],
                    "regressor__learning_rate": [0.01,0.1],
                },
                {
                    "regressor": [XGBRegressor()],
                    "regressor__n_estimators": [100,1000],
                    "regressor__learning_rate": [0.01,0.1],
                }
            ]
        }
    }
    return pipelines[type]["pipe"], [dict(pipelines[type]["grid_common"], **params) for params in pipelines[type]["grid_regressors"]]

In [ ]:
def get_trained_model(type, X_train, y_train, randomized=False):
    pipe, grid = get_pipe_grid(type)
    model = GridSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1) if not randomized else RandomizedSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1)
    model.fit(X_train, y_train)
    return model

In [ ]:
X_watt = details.drop(['power(watts)','distance(m)','altitude(m)','speed(m/s)','potenza_media','time_since_start','slope_percent','temperature(C)','hr_zone','pwr_zone','altitude_diff','distance_diff','left_pco(mm)','right_pco(mm)','power_left','power_right','accumulated_power(watts)'], axis=1)
y_watt = details['potenza_media']
X_train, X_val, y_train, y_val = train_test_split(X_watt, y_watt, test_size=1/3, random_state=42)

### Regressione con modelli lineari e polinomiali

In [ ]:
linear_gs = get_trained_model('linear', X_train, y_train)
print_eval(X_val, y_val, linear_gs)
linear_gs_res=pd.DataFrame(linear_gs.cv_results_).sort_values("mean_test_score", ascending=False)
linear_gs_res

### Regressione con funzioni kernel

In [ ]:
kernel_gs = get_trained_model('kernel', X_train, y_train)
print_eval(X_val, y_val, kernel_gs)
kernel_gs_res=pd.DataFrame(kernel_gs.cv_results_).sort_values("mean_test_score", ascending=False)
kernel_gs_res

### Regressione con alberi

In [ ]:
watt_tree_gs = get_trained_model('tree', X_train, y_train)
print_eval(X_val, y_val, watt_tree_gs, tree=True)
watt_tree_gs_res=pd.DataFrame(watt_tree_gs.cv_results_).sort_values("mean_test_score", ascending=False)
watt_tree_gs_res

### Confronto fra modelli

In [ ]:
# Creazione del plot scatter
fig, ax = plt.subplots(figsize=(24, 9))

linear_models = pd.concat([linear_gs_res, linear_gs_res], axis=0)
scatter_linear = ax.scatter(linear_models['mean_fit_time'], linear_models['mean_test_score'], color='blue', label='Modelli lineari e polinomiali')

krm_models = pd.concat([kernel_gs_res, kernel_gs_res], axis=0)
scatter_kernel = ax.scatter(krm_models['mean_fit_time'], krm_models['mean_test_score'], color='red', label='Modelli Kernel')

tree_models = pd.concat([watt_tree_gs_res, watt_tree_gs_res], axis=0)
scatter_tree = ax.scatter(tree_models['mean_fit_time'], tree_models['mean_test_score'], color='green', label='Modelli Albero')

ax.set_xlabel('Tempo di addestramento (s)')
ax.set_ylabel('Coefficiente R2')
ax.set_title('Confronto tra i modelli')
ax.legend()
plt.ylim(0, 1.1)
plt.show()


### test predizione

In [ ]:
X_watt.info()

In [ ]:
watt_tree_gs.predict(np.array([140,90]).reshape(1, -1))

### Reti neurali

In [ ]:
X_hr = details.drop(['heart_rate(bpm)','potenza_media','temperature(C)','hr_zone','pwr_zone','altitude_diff','distance_diff','left_pco(mm)','right_pco(mm)','power_left','power_right','accumulated_power(watts)'], axis=1)
y_hr = details['potenza_media']
X_train, X_val, y_train, y_val = train_test_split(X_hr, y_hr, test_size=1/3, random_state=42)

In [ ]:
from sklearn.neural_network import MLPRegressor

pipe = Pipeline([
    ("std", StandardScaler()),
    ("regressor", MLPRegressor())
])

grid = {
    "regressor__hidden_layer_sizes": [(256, 256)],
    "regressor__activation": ["relu"],
    "regressor__max_iter": [2000],
    "regressor__batch_size": [128],
    "regressor__alpha": [0.01]

}

model = GridSearchCV(pipe, grid, cv=kf, scoring="r2", n_jobs=-1)
model.fit(X_train, y_train)
print_eval(X_val, y_val, model)

In [ ]:
pd.DataFrame(model.cv_results_).sort_values("mean_test_score", ascending=False)

In [ ]:
pd.DataFrame({'Feature': X_watt.columns, 'Weight': model.best_estimator_.named_steps['regressor'].coefs_[0].mean(axis=1)})

# Classificazione

## Classificazione della potenza

In [ ]:
X = details[["time_since_start", "power(watts)"]]
y = details['pwr_zone']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

diagnosis_color_map = {1: "yellow", 2: "orange", 3: "red", 4: "purple", 5: "blue", 6: "green", 7: "black"}
X_train.plot.scatter("time_since_start", "power(watts)",c=y_train.map(diagnosis_color_map),figsize=(24, 6))

In [ ]:
zone_counts = details['pwr_zone'].value_counts()

zone_counts.plot.barh(figsize=(24, 6), legend=None)

plt.xlabel('tempo')
plt.ylabel('Zona')
plt.title('Conteggio del tempo a seconda delle zone')
plt.show()

In [ ]:
X = details[['heart_rate(bpm)', 'cadence(rpm)']]
y = details['pwr_zone']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1/3, random_state=42)

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)

plt.figure(figsize=(24, 9))
plot_tree(model, feature_names=X_train.columns.to_list())
print(export_text(model, feature_names=X_train.columns.to_list()))

## Classificazione dei battiti

In [ ]:
X = details[["time_since_start", "heart_rate(bpm)"]]
y = details['hr_zone']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

diagnosis_color_map = {1: "yellow", 2: "orange", 3: "red", 4: "purple", 5: "blue", 6: "green", 7: "black"}
X_train.plot.scatter("time_since_start", "heart_rate(bpm)",c=y_train.map(diagnosis_color_map),figsize=(24, 6))

In [ ]:
zone_counts = details['hr_zone'].value_counts()

zone_counts.plot.barh(figsize=(24, 6), legend=None)

plt.xlabel('tempo')
plt.ylabel('Zona')
plt.title('Conteggio del tempo a seconda delle zone')
plt.show()

In [ ]:
X = details[['power(watts)', 'cadence(rpm)']]
y = details['hr_zone']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=1/3, random_state=42)

model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)

plt.figure(figsize=(24, 9))
plot_tree(model, feature_names=X_train.columns.to_list())
print(export_text(model, feature_names=X_train.columns.to_list()))

# Flask

In [ ]:
import os
import pickle
import flask

In [ ]:
if not os.path.isdir("templates"):
  os.mkdir("templates")

if not os.path.isdir("models"):
  os.mkdir("models")

if not os.path.isdir("datasets"):
  os.mkdir("datasets")

In [ ]:
with open("models/hr_model.bin", "wb") as f:
    pickle.dump(tree_gs.best_estimator_, f)

with open("models/watt_model.bin", "wb") as f:
    pickle.dump(watt_tree_gs.best_estimator_, f)

with open("datasets/hr_dataset.pkl", "wb") as f:
    pickle.dump(X_hr, f)

with open("datasets/watt_dataset.pkl", "wb") as f:
    pickle.dump(X_watt, f)

In [ ]:
%%writefile garmin_flask.py
import os.path
import pickle
from flask import Flask, request, render_template
import numpy as np

In [ ]:
%%writefile -a garmin_flask.py

app = Flask(__name__)
app.debug = True

In [ ]:
%%writefile -a garmin_flask.py

@app.route("/", methods=["GET", "POST"])
def index():
    
    with open('datasets/hr_dataset.pkl', 'rb') as f:
        X_hr = pickle.load(f)
    with open('datasets/watt_dataset.pkl', 'rb') as f:
        X_watt = pickle.load(f)

    if request.method == "POST":
        card = request.form.get("card")

        if card == "hr":
            inputs = []
            for column_name, dtype in X_hr.dtypes.items():
                if dtype == "int64":
                    value = int(request.form[column_name])
                elif dtype == "float64":
                    value = float(request.form[column_name])
                inputs.append(value)
            with app.open_resource("models/hr_model.bin", "rb") as f:
                hr_model = pickle.load(f)
            response = hr_model.predict(np.array(inputs).reshape(1, -1))[0]
            return render_template("index.html", hr_pred=response, X_hr=X_hr, X_watt=X_watt)

        elif card == "watt":
            inputs = []
            for column_name, dtype in X_watt.dtypes.items():
                if dtype == "int64":
                    value = int(request.form[column_name])
                elif dtype == "float64":
                    value = float(request.form[column_name])
                inputs.append(value)
            with app.open_resource("models/watt_model.bin", "rb") as f:
                watt_model = pickle.load(f)
            response = watt_model.predict(np.array(inputs).reshape(1, -1))[0]
            return render_template("index.html", w_pred=response, X_hr=X_hr, X_watt=X_watt)

    return render_template("index.html", X_hr=X_hr, X_watt=X_watt)

In [ ]:
%%writefile -a garmin_flask.py

if __name__ == '__main__':
  app.run()

In [ ]:
%%writefile templates/index.html

<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Bootstrap demo</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet"
    integrity="sha384-9ndCyUaIbzAi2FUVXJi0CjmCapSmO7SnpJef0486qhLnuZ2cdeRhO02iuK6FUUVM" crossorigin="anonymous">
</head>

<body class="d-flex flex-column vh-100" data-bs-theme="dark">
  <header class="d-flex justify-content-center shadow py-3">
      <h1>data intensive</h1>
  </header>
  <div class="d-flex align-items-center justify-content-center h-100">
    <div class="card m-auto">
      <div class="card-body">
        <h5 class="card-title">Previsione battiti</h5>
        <form method="POST" action="">
          {% for column in X_hr.columns %}
          <div class="mb-3">
            <label for="{{ column }}" class="form-label">{{ column }}</label>
            <input name="{{ column }}" class="form-control" id="{{ column }}">
          </div>
          {% endfor %}
          <input type="hidden" name="card" value="hr">
          <button type="submit" class="btn btn-primary">Submit</button>
        </form>
      </div>
      <div class="card-footer">
        <p>Risultato: <b>{{ hr_pred }}</b></p>
      </div>
    </div>
    <div class="card m-auto">
      <div class="card-body">
        <h5 class="card-title">Previsione potenza</h5>
        <form method="POST" action="">
          {% for column in X_watt.columns %}
          <div class="mb-3">
            <label for="{{ column }}" class="form-label">{{ column }}</label>
            <input name="{{ column }}" class="form-control" id="{{ column }}">
          </div>
          {% endfor %}
          <input type="hidden" name="card" value="watt">
          <button type="submit" class="btn btn-primary">Submit</button>
        </form>
      </div>
      <div class="card-footer">
        <p>Risultato: <b>{{ w_pred }}</b></p>
      </div>
    </div>
  </div>
</body>

</html>
